In [ ]:
!pip install transformers torch  wandb tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from transformers import DistilBertTokenizer, DistilBertModel
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np


In [ ]:
# Load saved tensors
input_ids = torch.load("/content/drive/MyDrive/FYP Suicide Detection/DistilBERT/DistilBERTCNN/tokenized_tensors/input_ids.pt")
attention_mask = torch.load("/content/drive/MyDrive/FYP Suicide Detection/DistilBERT/DistilBERTCNN/tokenized_tensors/attention_mask.pt")
labels = torch.load("/content/drive/MyDrive/FYP Suicide Detection/DistilBERT/DistilBERTCNN/tokenized_tensors/labels.pt")


KeyboardInterrupt: 

In [ ]:
# Initialize Weights & Biases

#0228da7fe575258568d48818e3f64c2e74895a16

wandb.init(project="distilbert_cnn_v3", name="1epoch", resume="allow")

In [ ]:
#number 1

import torch
from torch.utils.data import Dataset, DataLoader
# ✅ Step 6: Define Custom Dataset class
class CustomDataset(Dataset):
    def __init__(self, input_ids, attention_mask, labels):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.labels[idx]
        }


In [ ]:
#number 2

# Assuming you've saved the tokenized tensors as .pt files
input_ids = torch.load('/content/drive/MyDrive/FYP Suicide Detection/DistilBERT/DistilBERTCNN/tokenized_tensors/input_ids.pt')
attention_mask = torch.load('/content/drive/MyDrive/FYP Suicide Detection/DistilBERT/DistilBERTCNN/tokenized_tensors/attention_mask.pt')
labels = torch.load('/content/drive/MyDrive/FYP Suicide Detection/DistilBERT/DistilBERTCNN/tokenized_tensors/labels.pt')

# ✅ Recreate the dataset with loaded tensors
dataset = CustomDataset(input_ids, attention_mask, labels)


In [ ]:
#Number 3

#when resuming the training
#run this before creating dataset
from torch.utils.data import Subset

# Load the saved split indices
train_indices, val_indices = torch.load('/content/drive/MyDrive/FYP Suicide Detection/DistilBERT/DistilBERTCNN/tokenized_tensors/split_indices.pt')

# Use Subset to create datasets with the saved indices
train_dataset = Subset(dataset, train_indices)
val_dataset = Subset(dataset, val_indices)

# Create DataLoaders
batch_size = 64  # or whatever you prefer
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [ ]:
model_checkpoint = "/content/drive/MyDrive/FYP Suicide Detection/DistilBERT/Text Model in Action/model checkpoints/2025-03-28_13-05-49/checkpoint-231840"

In [ ]:
class DistilBERT_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert = DistilBertModel.from_pretrained(config.model_checkpoint)
        self.convs = nn.ModuleList([
            nn.Conv1d(768, 128, kernel_size=k) for k in [2, 3, 4]  # Bi/tri/4-gram filters
        ])
        self.dropout = nn.Dropout(config.dropout)
        self.classifier = nn.Linear(128 * 3, 1)  # 3 filter groups

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = outputs.last_hidden_state.permute(0, 2, 1)  # (batch, 768, seq_len)

        conv_features = [torch.relu(conv(hidden_state)) for conv in self.convs]
        pooled = [torch.max(f, dim=2)[0] for f in conv_features]  # Global max pooling
        concat = self.dropout(torch.cat(pooled, dim=1))
        return self.classifier(concat)

In [ ]:
# Initialize Model & Optimizer
model = DistilBERT_CNN().to(config.device)
optimizer = torch.optim.AdamW([
    {'params': model.bert.parameters(), 'lr': config.bert_lr},
    {'params': model.convs.parameters(), 'lr': config.cnn_lr},
    {'params': model.classifier.parameters(), 'lr': config.cnn_lr}
])
criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([config.pos_weight]).to(config.device))
scaler = torch.cuda.amp.GradScaler()

In [ ]:
# 0228da7fe575258568d48818e3f64c2e74895a16
!wandb login --relogin

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


In [ ]:
import wandb

# Initialize W&B with project name, config, and tags
wandb.init(
    project="distilbert_cnn_v2",  # Your project name
    config={                       # Hyperparameters/dataset info
        "model": "DistilBERT + CNN",
        "batch_size": 64,
        "epochs": 10,
        "class_ratio": "60:40",
        "pos_weight": 1.5,
        "learning_rate_bert": 2e-5,
        "learning_rate_cnn": 1e-4
    },
    tags=["baseline", "hybrid"],  # Organize runs
    notes="Initial run with DistilBERT+CNN for suicide detection"  # Add context
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: muhib-57838 (muhib-57838-iqra-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
!pip install wandb transformers


NameError: name 'test_training' is not defined

In [ ]:
#Testing the wandb.
import torch
import wandb
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score
from transformers import DistilBertTokenizer, DistilBertModel
from torch.utils.data import Dataset, DataLoader

# Initialize W&B (test project)
wandb.init(project="distilbert_cnn_v2", name="dry-run", save_code=True)

# Synthetic data generator
class TestDataset(Dataset):
    def __init__(self, num_samples=100):
        self.texts = ["This is a test sentence."] * num_samples  # Dummy text
        self.labels = np.random.randint(0, 2, num_samples)  # Random 0/1 labels

        # Tokenize with DistilBERT
        self.tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
        self.encodings = self.tokenizer(
            self.texts, truncation=True, padding=True, max_length=128
        )

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return {
            "input_ids": torch.tensor(self.encodings["input_ids"][idx]),
            "attention_mask": torch.tensor(self.encodings["attention_mask"][idx]),
            "labels": torch.tensor(self.labels[idx]),
        }

# Minimal model (for testing)
class TestModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.bert = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.classifier = torch.nn.Linear(768, 1)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        return self.classifier(outputs.last_hidden_state[:, 0, :])

# Training loop with logging
def test_training():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = TestModel().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
    criterion = torch.nn.BCEWithLogitsLoss()

    # Synthetic data
    dataset = TestDataset(num_samples=16)  # Tiny batch
    loader = DataLoader(dataset, batch_size=4)

    for epoch in range(2):  # 2 epochs for testing
        model.train()
        all_preds, all_labels = [], []

        for batch in loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].float().to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask).squeeze()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # Simulate predictions
            preds = (torch.sigmoid(outputs) > 0.5).int().cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())

        # Calculate metrics
        accuracy = (np.array(all_preds) == np.array(all_labels)).mean()
        precision = precision_score(all_labels, all_preds, zero_division=0)
        recall = recall_score(all_labels, all_preds, zero_division=0)

        # Log to W&B
        wandb.log({
            "epoch": epoch,
            "loss": loss.item(),
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "confusion_matrix": wandb.plot.confusion_matrix(
                y_true=all_labels,
                preds=all_preds,
                class_names=["non_suicidal", "suicidal"]
            )
        })

    wandb.finish()

# Run the test
test_training()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: muhib-57838 (muhib-57838-iqra-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

accuracy,▁█
epoch,▁█
loss,▁█
precision,█▁
recall,█▁
accuracy,0.625
epoch,1
loss,0.72404
precision,0
recall,0


In [ ]:
# Cell 1: Install Dependencies
!pip install transformers torch wandb tqdm
# !wandb login


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
# Cell 2: Imports & Configuration
import torch
import wandb
import numpy as np
from torch import nn
from tqdm import tqdm
from transformers import DistilBertTokenizer, DistilBertModel
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

class Config:
    # Model
    model_checkpoint = "/content/drive/MyDrive/FYP Suicide Detection/DistilBERT/Text Model in Action/model checkpoints/2025-03-28_13-05-49/checkpoint-231840"
    pos_weight = 1.5  # 60:40 class ratio (60/40=1.5)

    # Training
    batch_size = 32
    max_seq_len = 256
    bert_lr = 2e-5
    cnn_lr = 1e-4
    epochs = 24
    max_grad_norm = 1.0
    dropout = 0.2

    checkpoint_dir = "/content/drive/MyDrive/FYP Suicide Detection/DistilBERT/DistilBERTCNN/trained_model_v3"
    resume_epoch = 20  # 👈 Specify which epoch to resume from

    # System
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    save_dir = "/content/drive/MyDrive/FYP Suicide Detection/DistilBERT/DistilBERTCNN/trained_model_v3/"

config = Config()



In [ ]:
# Cell 3: Model Architecture
class DistilBERT_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert = DistilBertModel.from_pretrained(config.model_checkpoint)
        self.convs = nn.ModuleList([
            nn.Conv1d(768, 128, kernel_size=k) for k in [2, 3, 4]
        ])
        self.dropout = nn.Dropout(config.dropout)
        self.classifier = nn.Linear(128 * 3, 1)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = outputs.last_hidden_state.permute(0, 2, 1)

        conv_features = [torch.relu(conv(hidden_state)) for conv in self.convs]
        pooled = [torch.max(f, dim=2)[0] for f in conv_features]
        concat = self.dropout(torch.cat(pooled, dim=1))
        return self.classifier(concat)



In [ ]:
# Initialize W&B (test project)
wandb.init(project="distilbert_cnn_v3", name="2-epochs", save_code=True)


In [ ]:
# Step 1: Create Dataset Class for Pre-Tokenized Data
class PreTokenizedDataset(torch.utils.data.Dataset):
    def __init__(self, input_ids, attention_mask, labels):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.labels[idx]
        }

    def __len__(self):
        return len(self.labels)

# Step 2: Load Your Saved Tensors
input_ids = torch.load('/content/drive/MyDrive/FYP Suicide Detection/DistilBERT/DistilBERTCNN/tokenized_tensors/input_ids.pt')
attention_mask = torch.load('/content/drive/MyDrive/FYP Suicide Detection/DistilBERT/DistilBERTCNN/tokenized_tensors/attention_mask.pt')
labels = torch.load('/content/drive/MyDrive/FYP Suicide Detection/DistilBERT/DistilBERTCNN/tokenized_tensors/labels.pt')

# Step 3: Create Full Dataset
full_dataset = PreTokenizedDataset(input_ids, attention_mask, labels)

# Step 4: Load Split Indices
train_indices, val_indices = torch.load('/content/drive/MyDrive/FYP Suicide Detection/DistilBERT/DistilBERTCNN/tokenized_tensors/split_indices.pt')

# Step 5: Create Train/Validation Subsets
train_dataset = Subset(full_dataset, train_indices)
val_dataset = Subset(full_dataset, val_indices)

# Step 6: Create DataLoaders
batch_size = 32  # Match your config
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    pin_memory=True  # Faster data transfer to GPU
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    pin_memory=True
)

# Step 7: Verify Data Shapes
sample_batch = next(iter(train_loader))
print("Sample batch:")
print(f"Input IDs shape: {sample_batch['input_ids'].shape}")
print(f"Attention mask shape: {sample_batch['attention_mask'].shape}")
print(f"Labels shape: {sample_batch['labels'].shape}")

# Expected output (for batch_size=32 and max_seq_len=128):
# Input IDs shape: torch.Size([32, 128])
# Attention mask shape: torch.Size([32, 128])
# Labels shape: torch.Size([32])

Sample batch:
Input IDs shape: torch.Size([32, 512])
Attention mask shape: torch.Size([32, 512])
Labels shape: torch.Size([32])


In [ ]:
# Cell 5: Training Loop with FP16
def train_model(train_loader, val_loader):
    # Initialize
    model = DistilBERT_CNN().to(config.device)
    optimizer = torch.optim.AdamW([
        {'params': model.bert.parameters(), 'lr': config.bert_lr},
        {'params': model.convs.parameters(), 'lr': config.cnn_lr},
        {'params': model.classifier.parameters(), 'lr': config.cnn_lr}
    ])
    criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([config.pos_weight]).to(config.device))
    scaler = torch.cuda.amp.GradScaler()
    best_val_f1 = 0.0

    # W&B Init
    wandb.init(project="distilbert_cn_v3", name = "2-4epochs" , config=vars(config),
              tags=["distilbert-cnn", "fp16"])

    for epoch in range(config.epochs):
        # Training
        model.train()
        train_preds, train_true = [], []

        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{config.epochs} [Train]"):
            input_ids = batch['input_ids'].to(config.device)
            attention_mask = batch['attention_mask'].to(config.device)
            labels = batch['labels'].float().to(config.device)

            optimizer.zero_grad()

            # Mixed precision forward
            with torch.cuda.amp.autocast():
                outputs = model(input_ids, attention_mask).squeeze()
                loss = criterion(outputs, labels)

            # Backward with gradient scaling
            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), config.max_grad_norm)
            scaler.step(optimizer)
            scaler.update()

            # Track metrics
            preds = (torch.sigmoid(outputs) > 0.5).cpu().float()
            train_preds.extend(preds.numpy())
            train_true.extend(labels.cpu().numpy())

        # Validation
        model.eval()
        val_preds, val_true = [], []
        with torch.no_grad():
            for batch in tqdm(val_loader, desc=f"Epoch {epoch+1}/{config.epochs} [Val]"):
                input_ids = batch['input_ids'].to(config.device)
                attention_mask = batch['attention_mask'].to(config.device)
                labels = batch['labels'].float().to(config.device)

                outputs = model(input_ids, attention_mask).squeeze()
                preds = (torch.sigmoid(outputs) > 0.5).cpu().float()
                val_preds.extend(preds.numpy())
                val_true.extend(labels.cpu().numpy())

        # Calculate Metrics
        def calculate_metrics(true, preds, prefix):
            return {
                f"{prefix}/loss": loss.item() if 'loss' in locals() else 0,
                f"{prefix}/accuracy": (np.array(true) == np.array(preds)).mean(),
                f"{prefix}/precision": precision_score(true, preds, average='binary', zero_division=0),
                f"{prefix}/recall": recall_score(true, preds, average='binary', zero_division=0),
                f"{prefix}/f1": f1_score(true, preds, average='binary', zero_division=0),
                f"{prefix}/cm": wandb.plot.confusion_matrix(
                    y_true=true,
                    preds=preds,
                    class_names=["non-suicidal", "suicidal"]
                )
            }

        # Log to W&B
        metrics = {
            "epoch": epoch + 1,
            **calculate_metrics(train_true, train_preds, "train"),
            **calculate_metrics(val_true, val_preds, "val")
        }
        wandb.log(metrics)

        # Save Best Model
        if metrics["val/f1"] > best_val_f1:
            best_val_f1 = metrics["val/f1"]
            torch.save(model.state_dict(), f"{config.save_dir}/best_model.pt")
            wandb.save(f"{config.save_dir}/best_model.pt")

    wandb.finish()



In [ ]:
#Saving the testing epoch 1 with
#failed to create an instance. double baam. issey naya object/instance bn jaiga.
checkpoint_path = "/content/drive/MyDrive/FYP Suicide Detection/DistilBERT/DistilBERTCNN/trained_model_v3/checkpoint_testing/"
model = DistilBERT_CNN().to(config.device)
optimizer = torch.optim.AdamW([
        {'params': model.bert.parameters(), 'lr': config.bert_lr},
        {'params': model.convs.parameters(), 'lr': config.cnn_lr},
        {'params': model.classifier.parameters(), 'lr': config.cnn_lr}
    ])
scaler = torch.cuda.amp.GradScaler()

checkpoint = {
        'epoch': 1,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scaler_state_dict': scaler.state_dict(),
        'best_f1': 0.9686  # Track but not used for saving
    }
torch.save(checkpoint, checkpoint_path)
print(f"💾 Saved checkpoint at epoch {1}")

NameError: name 'optimizer' is not defined

In [ ]:
checkpoint = {
    'epoch': 1,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'scaler_state_dict': scaler.state_dict(),
    'best_f1': best_f1
}
torch.save(checkpoint, checkpoint_path)


NameError: name 'optimizer' is not defined

In [ ]:
# Cell 5: Checkpoint Management Functions

def save_checkpoint(epoch, model, optimizer, scaler, config):
    checkpoint = {
        'epoch': epoch + 1,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scaler_state_dict': scaler.state_dict(),
        'best_f1': 0.9686  # Track but not used for saving
      }
    torch.save(checkpoint, checkpoint_path)
    print(f"💾 Saved checkpoint at epoch {epoch+1}")


In [ ]:
# Cell 5: Checkpoint Management Functions
def save_checkpoint(epoch, model, optimizer, scaler, config):
    checkpoint = {
        'epoch': epoch + 1,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scaler_state_dict': scaler.state_dict(),
        'best_f1': best_val_f1  # Track but not used for saving
    }
    torch.save(checkpoint, config.checkpoint_path)
    print(f"💾 Saved checkpoint at epoch {epoch+1}")

def load_checkpoint(config, model, optimizer, scaler):
    if os.path.exists(config.checkpoint_path):
        checkpoint = torch.load(config.checkpoint_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        scaler.load_state_dict(checkpoint['scaler_state_dict'])
        start_epoch = checkpoint['epoch']
        best_f1 = checkpoint['best_f1']
        print(f"🚀 Resuming from epoch {start_epoch} | Best F1: {best_f1:.4f}")
        return start_epoch, best_f1
    return 0, 0.0

# Cell 6: Enhanced Training Loop with Resume Support
def train_model(train_loader, val_loader):
    # Initialize model and optimizer
    model = DistilBERT_CNN().to(config.device)
    optimizer = torch.optim.AdamW([
        {'params': model.bert.parameters(), 'lr': config.bert_lr},
        {'params': model.convs.parameters(), 'lr': config.cnn_lr},
        {'params': model.classifier.parameters(), 'lr': config.cnn_lr}
    ])
    criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([config.pos_weight]).to(config.device))
    scaler = torch.cuda.amp.GradScaler()

    # Load checkpoint if exists
    start_epoch, best_val_f1 = load_checkpoint(config, model, optimizer, scaler)

    # W&B Initialization
    wandb.init(project="suicide-detection",
              config=vars(config),
              resume="allow",
              id="suicide-detection-run-1")  # Fixed run ID for resuming

    for epoch in range(start_epoch, config.epochs):
        # --- Training Phase ---
        model.train()
        train_preds, train_true = [], []
        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{config.epochs} [Train]"):
            # ... [existing training code] ...

        # --- Validation Phase ---
        model.eval()
        val_preds, val_true = [], []
        with torch.no_grad():
            # ... [existing validation code] ...

        # --- Metrics Calculation ---
        def calculate_metrics(true, preds, prefix):
            return {
                f"{prefix}/loss": criterion(torch.tensor(preds), torch.tensor(true)).item(),
                f"{prefix}/accuracy": (np.array(true) == np.array(preds)).mean(),
                f"{prefix}/precision": precision_score(true, preds, zero_division=0),
                f"{prefix}/recall": recall_score(true, preds, zero_division=0),
                f"{prefix}/f1": f1_score(true, preds, zero_division=0),
                f"{prefix}/cm": wandb.plot.confusion_matrix(
                    y_true=true,
                    preds=preds,
                    class_names=["non-suicidal", "suicidal"]
                )
            }

        # --- Logging & Checkpointing ---
        metrics = {
            "epoch": epoch + 1,
            **calculate_metrics(train_true, train_preds, "train"),
            **calculate_metrics(val_true, val_preds, "val")
        }
        wandb.log(metrics)

        # Save checkpoint every epoch
        save_checkpoint(epoch, model, optimizer, scaler, config)

    wandb.finish()

# Cell 7: Execute Training
if __name__ == "__main__":
    # Mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')

    # Create directories if needed
    os.makedirs(config.save_dir, exist_ok=True)

    # Start/resume training
    train_model(train_loader, val_loader)

In [ ]:
# Cell 6: Execute Training
if __name__ == "__main__":
    # # Mount Google Drive
    # from google.colab import drive
    # drive.mount('/content/drive')

    # Load your data here
    # train_texts, train_labels = ...
    # val_texts, val_labels = ...

    # # Create datasets
    # full_dataset = SuicideDataset(train_texts + val_texts, train_labels + val_labels)
    # train_dataset = Subset(full_dataset, train_indices)
    # val_dataset = Subset(full_dataset, val_indices)

    # # Create loaders
    # train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
    # val_loader = DataLoader(val_dataset, batch_size=config.batch_size)

    # Start training
    train_model(train_loader, val_loader)

<ipython-input-10-8dbaa8b6bbd9>:11: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


Epoch 1/1 [Train]:   0%|          | 0/5127 [00:00<?, ?it/s]<ipython-input-10-8dbaa8b6bbd9>:31: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch 1/1 [Val]: 100%|██████████| 570/570 [04:42<00:00,  2.02it/s]
wandb: WARNING Saving files without folders. If you want to preserve subdirectories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")


epoch,▁
train/accuracy,▁
train/f1,▁
train/loss,▁
train/precision,▁
train/recall,▁
val/accuracy,▁
val/f1,▁
val/loss,▁
val/precision,▁
val/recall,▁


In [ ]:
#0228da7fe575258568d48818e3f64c2e74895a16

In [ ]:
wandb.init(
        project="distilbert_cnn_v3",
        name="epoch_20-24_new",  # Short unique identifier
        config=vars(config),
        tags=["distilbert-cnn", "fp16", "production"],
        group="hybrid-model-v1",  # Group related experiments
        resume="allow"  # Critical for continuous logging
    )

In [ ]:
wandb.init(
        project="distilbert_cnn_v3",
        name="epoch_18-20",  # Short unique identifier
        config=vars(config),
        tags=["distilbert-cnn", "fp16", "production"],
        group="hybrid-model-v1",  # Group related experiments
        resume="allow"  # Critical for continuous logging
)


In [ ]:
import os

def train_model(train_loader, val_loader, resume=False):
    # Model init
    model = DistilBERT_CNN().to(config.device)

    # Optimizer
    optimizer = torch.optim.AdamW([
        {'params': model.bert.parameters(), 'lr': config.bert_lr},
        {'params': model.convs.parameters(), 'lr': config.cnn_lr},
        {'params': model.classifier.parameters(), 'lr': config.cnn_lr}
    ])

    # # W&B Configuration
    # run_id = None
    # if resume and os.path.exists(config.checkpoint_path):
    #     checkpoint = torch.load(config.checkpoint_path, map_location=config.device)
    #     run_id = checkpoint.get('wandb_run_id', wandb.util.generate_id())
    # else:
    #     run_id = wandb.util.generate_id()

    # Loss & AMP
    criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([config.pos_weight]).to(config.device))
    scaler = torch.cuda.amp.GradScaler()

    # Tracking
    # best_val_f1 = 0.0
    start_epoch = 20

    # Checkpoint path
    checkpoint_dir = config.save_dir
    os.makedirs(checkpoint_dir, exist_ok=True)

    # 💾 Resume checkpoint if available
    if resume:
        # Manually construct the target checkpoint filename
        target_checkpoint = f"checkpoint_epoch_{config.resume_epoch}.pt"
        checkpoint_path = os.path.join(config.checkpoint_dir, target_checkpoint)

        if not os.path.exists(checkpoint_path):
            raise FileNotFoundError(f"❌ Checkpoint for epoch {config.resume_epoch} not found!")

        checkpoint = torch.load(checkpoint_path, map_location=config.device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        scaler.load_state_dict(checkpoint['scaler_state_dict'])
        start_epoch = checkpoint['epoch']  # Will be 12 if resuming epoch 12
        best_val_f1 = checkpoint['best_f1']
        print(f"✅ Manually resumed from epoch {start_epoch} (loaded {target_checkpoint})")
    else:
        start_epoch = 0
        best_val_f1 = 0.0

        if not os.path.exists(checkpoint_path):
            raise FileNotFoundError(f"❌ Checkpoint for epoch {config.resume_epoch} not found!")

    for epoch in range(start_epoch, config.epochs):
        model.train()
        train_preds, train_true = [], []

        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{config.epochs} [Train]"):
            input_ids = batch['input_ids'].to(config.device)
            attention_mask = batch['attention_mask'].to(config.device)
            labels = batch['labels'].float().to(config.device)

            optimizer.zero_grad()

            with torch.cuda.amp.autocast():
                outputs = model(input_ids, attention_mask).squeeze()
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), config.max_grad_norm)
            scaler.step(optimizer)
            scaler.update()

            preds = (torch.sigmoid(outputs) > 0.5).cpu().float()
            train_preds.extend(preds.numpy())
            train_true.extend(labels.cpu().numpy())

        # Validation
        model.eval()
        val_preds, val_true = [], []

        with torch.no_grad():
            for batch in tqdm(val_loader, desc=f"Epoch {epoch+1}/{config.epochs} [Val]"):
                input_ids = batch['input_ids'].to(config.device)
                attention_mask = batch['attention_mask'].to(config.device)
                labels = batch['labels'].float().to(config.device)

                outputs = model(input_ids, attention_mask).squeeze()
                preds = (torch.sigmoid(outputs) > 0.5).cpu().float()
                val_preds.extend(preds.numpy())
                val_true.extend(labels.cpu().numpy())

        # Metrics
        def calculate_metrics(true, preds, prefix):
            return {
                f"{prefix}/loss": loss.item(),
                f"{prefix}/accuracy": (np.array(true) == np.array(preds)).mean(),
                f"{prefix}/precision": precision_score(true, preds, average='binary', zero_division=0),
                f"{prefix}/recall": recall_score(true, preds, average='binary', zero_division=0),
                f"{prefix}/f1": f1_score(true, preds, average='binary', zero_division=0),
                f"{prefix}/cm": wandb.plot.confusion_matrix(
                    y_true=true,
                    preds=preds,
                    class_names=["non-suicidal", "suicidal"]
                )
            }

        metrics = {
            "epoch": epoch + 1,
            **calculate_metrics(train_true, train_preds, "train"),
            **calculate_metrics(val_true, val_preds, "val")
        }
        wandb.log(metrics)

        # # Save best model (optional)
        # if metrics["val/f1"] > best_val_f1:
        #     best_val_f1 = metrics["val/f1"]
        #     torch.save(model.state_dict(), os.path.join(checkpoint_dir, "best_model.pt"))
        #     wandb.save(os.path.join(checkpoint_dir, "best_model.pt"))
        # 💾 Save checkpoint for this epoch

        checkpoint = {
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scaler_state_dict': scaler.state_dict(),
            'best_f1': best_val_f1
        }
        torch.save(checkpoint, os.path.join(checkpoint_dir, f"checkpoint_epoch_{epoch+1}.pt"))
        print(f"💾 Saved checkpoint for epoch {epoch+1}")

    wandb.finish()


In [ ]:
train_model(train_loader, val_loader, resume=True)


<ipython-input-14-8e1ef5fb326c>:24: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


✅ Manually resumed from epoch 20 (loaded checkpoint_epoch_20.pt)


Epoch 21/24 [Train]:   0%|          | 0/5127 [00:00<?, ?it/s]<ipython-input-14-8e1ef5fb326c>:68: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch 21/24 [Val]: 100%|██████████| 570/570 [04:29<00:00,  2.12it/s]


💾 Saved checkpoint for epoch 21


Epoch 22/24 [Train]:   0%|          | 0/5127 [00:00<?, ?it/s]<ipython-input-14-8e1ef5fb326c>:68: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch 22/24 [Val]: 100%|██████████| 570/570 [04:28<00:00,  2.12it/s]


...

Epoch 23/24 [Train]:   0%|          | 0/5127 [00:00<?, ?it/s]<ipython-input-14-8e1ef5fb326c>:68: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch 23/24 [Val]: 100%|██████████| 570/570 [04:29<00:00,  2.11it/s]


💾 Saved checkpoint for epoch 23


Epoch 24/24 [Train]:   0%|          | 0/5127 [00:00<?, ?it/s]<ipython-input-14-8e1ef5fb326c>:68: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch 24/24 [Val]: 100%|██████████| 570/570 [04:29<00:00,  2.12it/s]


💾 Saved checkpoint for epoch 24


epoch,▁▃▆█
train/accuracy,▁▆██
train/f1,▁▆██
train/loss,█▁▁▂
train/precision,▁▃▄█
train/recall,▁▇█▃
val/accuracy,▅█▅▁
val/f1,▂█▅▁
val/loss,█▁▁▂
val/precision,█▄▃▁
val/recall,▁▆▇█


In [ ]:
/content/drive/MyDrive/FYP Suicide Detection/DistilBERT/DistilBERTCNN/trained_model_v3/checkpoint_epoch_4.pt

In [ ]:
# Cell 10: Validation Loss Calculator
def calculate_validation_loss(checkpoint_path, val_loader, device, criterion):
    """Calculate and print only validation loss for a checkpoint"""
    # Initialize model
    model = DistilBERT_CNN().to(device)

    # Load checkpoint
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()

    # Calculate loss
    total_loss = 0.0
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Calculating Loss"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].float().to(device)

            outputs = model(input_ids, attention_mask).squeeze()
            loss = criterion(outputs, labels)
            total_loss += loss.item()

    avg_loss = total_loss / len(val_loader)
    print(f"\n🔍 Validation Loss for {checkpoint_path}:")
    print(f"- Loss: {avg_loss:.4f}")

# Cell 11: Example Usage
if __name__ == "__main__":
    # Initialize criterion with same parameters as training
    criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([config.pos_weight]).to(config.device))

    # Checkpoint to evaluate
    checkpoint_path = "/content/drive/MyDrive/FYP Suicide Detection/DistilBERT/DistilBERTCNN/trained_model_v3/checkpoint_epoch_7.pt"

    # Calculate and print only loss
    calculate_validation_loss(
        checkpoint_path=checkpoint_path,
        val_loader=val_loader,
        device=config.device,
        criterion=criterion
    )

Calculating Loss: 100%|██████████| 570/570 [04:22<00:00,  2.17it/s]


🔍 Validation Loss for /content/drive/MyDrive/FYP Suicide Detection/DistilBERT/DistilBERTCNN/trained_model_v3/checkpoint_epoch_7.pt:
- Loss: 0.1909


In [ ]:
import torch
import wandb
import numpy as np
from torch import nn
from tqdm import tqdm
from transformers import DistilBertTokenizer, DistilBertModel
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

In [ ]:
#Improved version of code !!!!!!!!!!!!!!!!!
# Cell 10: Validation Loss Calculator (Improved)

def validate_model(checkpoint_path, val_loader, device, criterion):
    """Validate a checkpoint with full metrics including loss"""
    # Initialize model
    model = DistilBERT_CNN().to(device)

    # Load checkpoint
    if not os.path.exists(checkpoint_path):
        raise FileNotFoundError(f"Checkpoint {checkpoint_path} not found!")

    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()

    # Initialize metrics
    total_loss = 0.0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].float().to(device)

            # Forward pass
            outputs = model(input_ids, attention_mask).squeeze()

            # Calculate loss
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            # Get predictions
            preds = (torch.sigmoid(outputs) > 0.5).cpu().float()
            all_preds.extend(preds.numpy())
            all_labels.extend(labels.cpu().numpy())

    # Calculate metrics
    avg_loss = total_loss / len(val_loader)
    accuracy = (np.array(all_labels) == np.array(all_preds)).mean()
    precision = precision_score(all_labels, all_preds, zero_division=0)
    recall = recall_score(all_labels, all_preds, zero_division=0)
    f1 = f1_score(all_labels, all_preds, zero_division=0)
    cm = confusion_matrix(all_labels, all_preds)

    # Print comprehensive results
    print("\n📊 Complete Validation Metrics:")
    print(f"- Loss:      {avg_loss:.4f}")
    print(f"- Accuracy:  {accuracy:.4f}")
    print(f"- Precision: {precision:.4f}")
    print(f"- Recall:    {recall:.4f}")
    print(f"- F1 Score:  {f1:.4f}")
    print("\n🔍 Confusion Matrix:")
    print(f"               Predicted 0 (Non-Suicidal)  Predicted 1 (Suicidal)")
    print(f"Actual 0 (Non-Suicidal) {cm[0,0]:>18} {cm[0,1]:>18}")
    print(f"Actual 1 (Suicidal)     {cm[1,0]:>18} {cm[1,1]:>18}")

# Cell 11: Example Usage with Full Metrics
if __name__ == "__main__":
    # Initialize criterion with same parameters as training
    criterion = nn.BCEWithLogitsLoss(
        pos_weight=torch.tensor([config.pos_weight]).to(config.device)
    )

    # # Validate best model
    # best_model_path = f"{config.save_dir}/best_model.pt"
    # validate_model(best_model_path, val_loader, config.device, criterion)
    model = DistilBERT_CNN().to(config.device)

    # Validate specific checkpoint
    epoch_checkpoint = f"{config.save_dir}/checkpoint_epoch_24.pt"
    validate_model(epoch_checkpoint, val_loader, config.device, criterion)

Validating: 100%|██████████| 570/570 [04:29<00:00,  2.12it/s]


📊 Complete Validation Metrics:
- Loss:      0.2488
- Accuracy:  0.9745
- Precision: 0.9673
- Recall:    0.9690
- F1 Score:  0.9682

🔍 Confusion Matrix:
               Predicted 0 (Non-Suicidal)  Predicted 1 (Suicidal)
Actual 0 (Non-Suicidal)              10692                239
Actual 1 (Suicidal)                    226               7071


In [ ]:
# Cell 12: Model Testing with Confidence Scores
def test_sample_statements(model, tokenizer, device, statements, threshold=0.5):
    """Test the model on custom statements with confidence metrics"""
    # Prepare model for inference
    model.eval()

    # Tokenize inputs
    encodings = tokenizer(statements,
                         padding=True,
                         truncation=True,
                         max_length=256,
                         return_tensors="pt")

    # Move to device
    input_ids = encodings['input_ids'].to(device)
    attention_mask = encodings['attention_mask'].to(device)

    # Get predictions
    with torch.no_grad():
        outputs = model(input_ids, attention_mask).squeeze()
        probabilities = torch.sigmoid(outputs).cpu().numpy()

    # Generate predictions and confidence metrics
    results = []
    for text, prob in zip(statements, probabilities):
        prediction = 1 if prob >= threshold else 0
        confidence = prob if prediction == 1 else 1 - prob
        class_0_prob = 1 - prob
        class_1_prob = prob

        results.append({
            "text": text,
            "prediction": "Suicidal" if prediction == 1 else "Non-Suicidal",
            "confidence": f"{confidence:.1%}",
            "probability_distribution": {
                "Non-Suicidal": f"{class_0_prob:.3f}",
                "Suicidal": f"{class_1_prob:.3f}"
            },
            "threshold_used": threshold,
            "raw_output": f"{prob:.4f}"
        })

    # Print formatted results
    print("\n🔍 Model Predictions with Confidence Scores:")
    for idx, result in enumerate(results, 1):
        print(f"\n📄 Sample {idx}:")
        print(f"- Text: {result['text']}")
        print(f"- Prediction: {result['prediction']} (Confidence: {result['confidence']})")
        print("- Class Probabilities:")
        print(f"  Non-Suicidal: {result['probability_distribution']['Non-Suicidal']}")
        print(f"  Suicidal:     {result['probability_distribution']['Suicidal']}")
        print(f"- Raw Output: {result['raw_output']} (Threshold: {result['threshold_used']})")

    return results

# Cell 13: Example Usage
if __name__ == "__main__":
    # Load components
    model = DistilBERT_CNN().to(config.device)
    checkpoint = torch.load(
        f"{config.save_dir}/checkpoint_epoch_24.pt",
        map_location=config.device
    )
    model.load_state_dict(checkpoint['model_state_dict'])  # Key fix here
    tokenizer = DistilBertTokenizer.from_pretrained(config.model_checkpoint)

    # Test samples
    test_samples = [
        "I feel hopeless and want to end it all.",
        "Today was the best day of my life!",
        "Sometimes I wonder if anyone would care if I disappeared.",
        "Just finished a great workout, feeling amazing!",
        "The pain is too much to bear anymore."
    ]

    # Get predictions
    predictions = test_sample_statements(model, tokenizer, config.device, test_samples)


🔍 Model Predictions with Confidence Scores:

📄 Sample 1:
- Text: I feel hopeless and want to end it all.
- Prediction: Suicidal (Confidence: 100.0%)
- Class Probabilities:
  Non-Suicidal: 0.000
  Suicidal:     1.000
- Raw Output: 1.0000 (Threshold: 0.5)

📄 Sample 2:
- Text: Today was the best day of my life!
- Prediction: Non-Suicidal (Confidence: 87.1%)
- Class Probabilities:
  Non-Suicidal: 0.871
  Suicidal:     0.129
- Raw Output: 0.1285 (Threshold: 0.5)

📄 Sample 3:
- Text: Sometimes I wonder if anyone would care if I disappeared.
- Prediction: Suicidal (Confidence: 100.0%)
- Class Probabilities:
  Non-Suicidal: 0.000
  Suicidal:     1.000
- Raw Output: 1.0000 (Threshold: 0.5)

📄 Sample 4:
- Text: Just finished a great workout, feeling amazing!
- Prediction: Non-Suicidal (Confidence: 100.0%)
- Class Probabilities:
  Non-Suicidal: 1.000
  Suicidal:     0.000
- Raw Output: 0.0000 (Threshold: 0.5)

📄 Sample 5:
- Text: The pain is too much to bear anymore.
- Prediction: Suicidal (Conf

In [ ]:

# Test samples
test_samples = [
  "I feel hopeless and want to end it all.",
  "Today was the best day of my life!",
  "Sometimes I wonder if anyone would care if I disappeared.",
  "Just finished a great workout, feeling amazing!",
  "The pain is too much to bear anymore.",
  "after one year of collecting these pieces of art, i present to you my collection.",
  "i am spenttldr i have a hard life better than many, worse than other and it does not seem to get better. i am out of hope and am really close to giving up. to me, my life has always been shit. my dad ditched the family when i was five. my mom dated a string of abusive boyfriends one of them analy and orally raped me when i was 8, then threatened to have his friends rape and murder my entire family while i watched if i told anyone then she quickly graduated from alcoholism to meth addiction. we were evicted from almost every place we lived in, then my brother and i were taken in by a family we knew from church for about a year until they gave up on us because my mom, who at this point had ditched us to live in vegas with her methead boyfriend, refused to sign over custody of us to them. despite all this, i grew up with hope that somehow i could make my life better. i did terrible in school with a gpa of maybe 1.2 because i missed so many days, never did my homework, and just fucking hated life because i was always hungry, did not fit in with anyone, and was told all my life how ugly and fat i was because everyone else in my family was rail thin and short while i was tall and built like a linebacker. after highschool, i joined the military because it was the only way i could see getting out of poverty. my recruiter was the first person to pretend to believe in me and broke regulation to pay for my adult education classes so i could actually get a high school diploma to join it was 1999 at the time. i actually enjoyed the military and did well. it is wild what 3 meals a day and some actual structure will do for your life. i graduated in the top 10 and went to my first duty assignment overseas. we were at peace then, and things were going really well. i started taking college courses and met the love of my life during carnevale. then 911 happened. i deployed first to iraq in 2003 and saw a lot of shit. i was in a near ambush which wounded many of my pals and killed my best friend at the time.",
  "sometimes I wish I had to kill someone using knife."

]

# Get predictions
predictions = test_sample_statements(model, tokenizer, config.device, test_samples)


🔍 Model Predictions with Confidence Scores:

📄 Sample 1:
- Text: I feel hopeless and want to end it all.
- Prediction: Suicidal (Confidence: 100.0%)
- Class Probabilities:
  Non-Suicidal: 0.000
  Suicidal:     1.000
- Raw Output: 1.0000 (Threshold: 0.5)

📄 Sample 2:
- Text: Today was the best day of my life!
- Prediction: Non-Suicidal (Confidence: 87.1%)
- Class Probabilities:
  Non-Suicidal: 0.871
  Suicidal:     0.129
- Raw Output: 0.1287 (Threshold: 0.5)

📄 Sample 3:
- Text: Sometimes I wonder if anyone would care if I disappeared.
- Prediction: Suicidal (Confidence: 100.0%)
- Class Probabilities:
  Non-Suicidal: 0.000
  Suicidal:     1.000
- Raw Output: 1.0000 (Threshold: 0.5)

📄 Sample 4:
- Text: Just finished a great workout, feeling amazing!
- Prediction: Non-Suicidal (Confidence: 100.0%)
- Class Probabilities:
  Non-Suicidal: 1.000
  Suicidal:     0.000
- Raw Output: 0.0000 (Threshold: 0.5)

📄 Sample 5:
- Text: The pain is too much to bear anymore.
- Prediction: Suicidal (Conf

In [ ]:
# Cell 12: Model Testing with Confidence Scores
def test_sample_statements(model, tokenizer, device, statements, threshold=0.5):
    """Test the model on custom statements with confidence metrics"""
    # Prepare model for inference
    model.eval()

    # Tokenize inputs
    encodings = tokenizer(statements,
                         padding=True,
                         truncation=True,
                         max_length=256,
                         return_tensors="pt")

    # Move to device
    input_ids = encodings['input_ids'].to(device)
    attention_mask = encodings['attention_mask'].to(device)

    # Get predictions
    with torch.no_grad():
        outputs = model(input_ids, attention_mask).squeeze()
        probabilities = torch.sigmoid(outputs).cpu().numpy()

    # Generate predictions and confidence metrics
    results = []
    for text, prob in zip(statements, probabilities):
        prediction = 1 if prob >= threshold else 0
        confidence = prob if prediction == 1 else 1 - prob
        class_0_prob = 1 - prob
        class_1_prob = prob

        results.append({
            "text": text,
            "prediction": "Suicidal" if prediction == 1 else "Non-Suicidal",
            "confidence": f"{confidence:.1%}",
            "probability_distribution": {
                "Non-Suicidal": f"{class_0_prob:.3f}",
                "Suicidal": f"{class_1_prob:.3f}"
            },
            "threshold_used": threshold,
            "raw_output": f"{prob:.4f}"
        })

    # Print formatted results
    print("\n🔍 Model Predictions with Confidence Scores:")
    for idx, result in enumerate(results, 1):
        print(f"\n📄 Sample {idx}:")
        print(f"- Text: {result['text']}")
        print(f"- Prediction: {result['prediction']} (Confidence: {result['confidence']})")
        print("- Class Probabilities:")
        print(f"  Non-Suicidal: {result['probability_distribution']['Non-Suicidal']}")
        print(f"  Suicidal:     {result['probability_distribution']['Suicidal']}")
        print(f"- Raw Output: {result['raw_output']} (Threshold: {result['threshold_used']})")

    return results

# Cell 13: Example Usage
if __name__ == "__main__":
    # Load components
    model = DistilBERT_CNN().to(config.device)
    checkpoint = torch.load(
        f"{config.save_dir}/checkpoint_epoch_24.pt",
        map_location=config.device
    )
    model.load_state_dict(checkpoint['model_state_dict'])  # Key fix here
    tokenizer = DistilBertTokenizer.from_pretrained(config.model_checkpoint)

    # Test samples
    test_samples = [
        "I feel hopeless and want to end it all.",
        "Today was the best day of my life!",
        "Sometimes I wonder if anyone would care if I disappeared.",
        "Just finished a great workout, feeling amazing!",
        "The pain is too much to bear anymore."
    ]

    # Get predictions
    predictions = test_sample_statements(model, tokenizer, config.device, test_samples)


🔍 Model Predictions with Confidence Scores:

📄 Sample 1:
- Text: I feel hopeless and want to end it all.
- Prediction: Suicidal (Confidence: 100.0%)
- Class Probabilities:
  Non-Suicidal: 0.000
  Suicidal:     1.000
- Raw Output: 1.0000 (Threshold: 0.5)

📄 Sample 2:
- Text: Today was the best day of my life!
- Prediction: Non-Suicidal (Confidence: 87.1%)
- Class Probabilities:
  Non-Suicidal: 0.871
  Suicidal:     0.129
- Raw Output: 0.1285 (Threshold: 0.5)

📄 Sample 3:
- Text: Sometimes I wonder if anyone would care if I disappeared.
- Prediction: Suicidal (Confidence: 100.0%)
- Class Probabilities:
  Non-Suicidal: 0.000
  Suicidal:     1.000
- Raw Output: 1.0000 (Threshold: 0.5)

📄 Sample 4:
- Text: Just finished a great workout, feeling amazing!
- Prediction: Non-Suicidal (Confidence: 100.0%)
- Class Probabilities:
  Non-Suicidal: 1.000
  Suicidal:     0.000
- Raw Output: 0.0000 (Threshold: 0.5)

📄 Sample 5:
- Text: The pain is too much to bear anymore.
- Prediction: Suicidal (Conf

In [ ]:
# Test samples
test_samples = [
    "I woke up this morning and started my day like usual—brushed my teeth, made a cup of coffee, sat by the window watching the street fill up with life, people hurrying to work, kids with backpacks rushing to catch the school bus, all of it so routine and uneventful that I didn’t even notice the time pass. It’s one of those days where nothing feels significant, nothing feels wrong either, but nothing excites me either. It’s just a day like any other, and that’s okay. Sometimes normalcy is a relief. Not everything has to be dramatic to be real.",
    "Today was one of those rare days where everything just clicked—I finished my work early, caught up with an old friend over lunch, and took a long walk as the sun was setting, painting the sky in orange and purple hues. The cool breeze, the laughter, the tiny surprises throughout the day reminded me that joy doesn’t always come in big moments. It often sneaks in through quiet contentment, through shared smiles and peace in your chest.",
    "Lately I’ve been feeling this heaviness I can’t quite explain. I get up, eat, work, sleep—but it all feels so empty. I don’t cry, I don’t scream, but there’s this dull ache under the surface, like I’m carrying a weight I can’t put down. I don’t want to alarm anyone, because I know it’s temporary. I’ve been here before and made it out, but this time it’s lonelier. I’m not thinking about ending things—I’m just tired. Tired of pretending I’m okay when I’m not, tired of smiling just so people don’t ask what’s wrong. I just want to feel like myself again.",
    "There’s a part of me that wants to scream, but the other part stays silent because it’s safer that way. I walk through life wearing a mask, laughing at jokes that don’t land, pretending the hollow space inside me isn’t expanding by the day. No one notices the subtle shifts, the way I disappear even when I’m present. I think about what it would feel like to not feel anything at all. I’m not saying I want to die. I just want the pain to stop, to wake up one day and not dread the hours ahead. Is that too much to ask?",
    "Every night I lie awake thinking about how much easier everything would be if I just closed my eyes and never had to wake up again. I’ve tried reaching out, I’ve tried therapy, medication, prayer, everything people tell you to try when you’re drowning—but nothing changes. It’s like the sadness has wrapped itself around my bones. I don’t want to keep pretending I’m okay when I’m not. I don’t want to keep being strong when every cell in my body begs me to let go. Maybe people will be sad for a while, but eventually they’ll move on. They always do."

]

    # Get predictions
predictions = test_sample_statements(model, tokenizer, config.device, test_samples)


🔍 Model Predictions with Confidence Scores:

📄 Sample 1:
- Text: I woke up this morning and started my day like usual—brushed my teeth, made a cup of coffee, sat by the window watching the street fill up with life, people hurrying to work, kids with backpacks rushing to catch the school bus, all of it so routine and uneventful that I didn’t even notice the time pass. It’s one of those days where nothing feels significant, nothing feels wrong either, but nothing excites me either. It’s just a day like any other, and that’s okay. Sometimes normalcy is a relief. Not everything has to be dramatic to be real.
- Prediction: Non-Suicidal (Confidence: 99.9%)
- Class Probabilities:
  Non-Suicidal: 0.999
  Suicidal:     0.001
- Raw Output: 0.0007 (Threshold: 0.5)

📄 Sample 2:
- Text: Today was one of those rare days where everything just clicked—I finished my work early, caught up with an old friend over lunch, and took a long walk as the sun was setting, painting the sky in orange and purple h

In [ ]:
#Testing on another dataset

# Cell 12: Twitter Dataset Evaluator
def evaluate_twitter_dataset(checkpoint_path, tweet_csv_path, device):
    """
    Evaluate model on Twitter dataset with comprehensive metrics
    Args:
        checkpoint_path: Path to model checkpoint
        tweet_csv_path: Path to CSV with 'text' and 'label' columns
        device: torch.device ('cuda' or 'cpu')
    Returns:
        Dictionary containing all evaluation metrics
    """
    # 1. Load Model
    model = DistilBERT_CNN().to(device)
    model.load_state_dict(torch.load(checkpoint_path, map_location=device)['model_state_dict'])
    model.eval()

    # 2. Prepare Dataset (Matching Training Preprocessing)
    tokenizer = DistilBertTokenizer.from_pretrained(config.model_checkpoint)

    class TweetDataset(Dataset):
        def __init__(self, texts, labels):
            self.encodings = tokenizer(
                texts,
                padding=True,
                truncation=True,
                max_length=256,
                return_tensors="pt"
            )
            self.labels = labels

        def __getitem__(self, idx):
            return {
                'input_ids': self.encodings['input_ids'][idx],
                'attention_mask': self.encodings['attention_mask'][idx],
                'labels': torch.tensor(self.labels[idx])
            }

        def __len__(self):
            return len(self.labels)

    # Load and prepare data
    tweet_csv_path = "/content/drive/MyDrive/FYP Suicide Detection/DistilBERT/datasets/twitter_labelled_v1.csv"
    df = pd.read_csv(tweet_csv_path)
    dataset = TweetDataset(df['text'].tolist(), df['label'].values)
    loader = DataLoader(dataset, batch_size=32, shuffle=False)

    # 3. Evaluation Metrics
    criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([config.pos_weight]).to(device))
    results = {
        'pred_probs': [],
        'pred_labels': [],
        'true_labels': []
    }
    total_loss = 0.0

    with torch.no_grad():
        for batch in tqdm(loader, desc="Evaluating Tweets"):
            inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
            labels = batch['labels'].float().to(device)

            outputs = model(**inputs).squeeze()
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            probs = torch.sigmoid(outputs).cpu().numpy()
            preds = (probs > 0.5).astype(int)

            results['pred_probs'].extend(probs)
            results['pred_labels'].extend(preds)
            results['true_labels'].extend(labels.cpu().numpy())

    # 4. Calculate Metrics
    metrics = {
        'loss': total_loss / len(loader),
        'accuracy': accuracy_score(results['true_labels'], results['pred_labels']),
        'precision': precision_score(results['true_labels'], results['pred_labels'], zero_division=0),
        'recall': recall_score(results['true_labels'], results['pred_labels'], zero_division=0),
        'f1': f1_score(results['true_labels'], results['pred_labels'], zero_division=0),
        'confusion_matrix': confusion_matrix(results['true_labels'], results['pred_labels']),
        'classification_report': classification_report(
            results['true_labels'],
            results['pred_labels'],
            target_names=['Non-Suicidal', 'Suicidal'])
    }

    # 5. Visualization-Ready Outputs
    print("\n🔥 Twitter Dataset Evaluation 🔥")
    print(f"Model: {os.path.basename(checkpoint_path)}")
    print(f"Samples: {len(results['true_labels'])}")
    print(f"\n📊 Key Metrics:")
    print(f"- Loss: {metrics['loss']:.4f}")
    print(f"- Accuracy: {metrics['accuracy']:.4f}")
    print(f"- Precision: {metrics['precision']:.4f}")
    print(f"- Recall: {metrics['recall']:.4f}")
    print(f"- F1: {metrics['f1']:.4f}")

    print("\n🧮 Confusion Matrix:")
    print(pd.DataFrame(
        metrics['confusion_matrix'],
        index=['Actual Non-Suicidal', 'Actual Suicidal'],
        columns=['Predicted Non-Suicidal', 'Predicted Suicidal']
    ))

    print("\n📝 Classification Report:")
    print(metrics['classification_report'])

    return metrics, results

# Cell 13: Execution Example
if __name__ == "__main__":
    # Configuration
    config = Config()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Paths (update these)
    checkpoint_path = f"{config.save_dir}/checkpoint_epoch_24.pt"
    tweet_csv_path = "path/to/twitter_dataset.csv"

    # Run evaluation
    metrics, detailed_results = evaluate_twitter_dataset(
        checkpoint_path=checkpoint_path,
        tweet_csv_path=tweet_csv_path,
        device=device
    )

    # Optional: Save results
    pd.DataFrame({
        'text': pd.read_csv(tweet_csv_path)['text'],
        'true_label': detailed_results['true_labels'],
        'pred_label': detailed_results['pred_labels'],
        'pred_prob': detailed_results['pred_probs']
    }).to_csv("twitter_predictions.csv", index=False)